In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from deap import base, creator, tools, algorithms
import random


In [2]:
df = pd.read_csv('titanic.csv')


In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.shape

(891, 12)

In [5]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [6]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [7]:
df['Age'].fillna(df['Age'].median(), inplace=True)
df['CabinKnown'] = df['Cabin'].notnull().astype(int)
df['Cabin'].fillna('Unknown', inplace=True)

C:\Users\varun\AppData\Local\Temp\ipykernel_11240\136996676.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].median(), inplace=True)
C:\Users\varun\AppData\Local\Temp\ipykernel_11240\136996676.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exa

In [8]:
df.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       2
CabinKnown     0
dtype: int64

In [9]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'CabinKnown'],
      dtype='object')

In [10]:
df = df[["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked", "Survived"]]

In [11]:
df.dropna(inplace=True)


In [12]:
label_enc = LabelEncoder()
df.loc[:, "Sex"] = label_enc.fit_transform(df["Sex"])
df.loc[:, "Embarked"] = label_enc.fit_transform(df["Embarked"])

In [13]:
X = df.drop("Survived", axis=1)
y = df["Survived"]

In [14]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2,random_state=42)

In [16]:

# Set up DEAP
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

In [17]:
toolbox = base.Toolbox()

In [18]:
import random
from deap import base, creator, tools

# Define fitness and individual
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

# Attribute generators
toolbox.register("n_estimators", random.randint, 10, 200)
toolbox.register("max_depth", random.randint, 3, 20)
toolbox.register("min_samples_split", random.randint, 2, 10)

# Structure initializers
toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.n_estimators, toolbox.max_depth, toolbox.min_samples_split), n=1)

toolbox.register("population", tools.initRepeat, list, toolbox.individual)


C:\Users\varun\anaconda3\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\varun\anaconda3\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [19]:
# Fitness function
def evaluate(ind):
 clf = RandomForestClassifier(
 n_estimators=ind[0],
 max_depth=ind[1],
 min_samples_split=ind[2],
 random_state=42
 )
 clf.fit(X_train, y_train)
 preds = clf.predict(X_test)
 return (accuracy_score(y_test, preds),)

In [20]:
# Evaluation function should be defined separately as `evaluate(individual)`
toolbox.register("evaluate", evaluate)

# Crossover: Uniform crossover with independent probability per gene
toolbox.register("mate", tools.cxUniform, indpb=0.5)

# Mutation: Uniform integer mutation within specified bounds
toolbox.register("mutate", tools.mutUniformInt,
                 low=[10, 3, 2],         # Lower bounds for [n_estimators, max_depth, min_samples_split]
                 up=[200, 20, 10],       # Upper bounds for each gene
                 indpb=0.2)              # Probability to mutate each gene

# Selection: Tournament selection
toolbox.register("select", tools.selTournament, tournsize=3)


In [21]:
# Run GA
def run_deap():
    pop = toolbox.population(n=10)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("max", np.max)

    pop, logbook = algorithms.eaSimple(
        population=pop,
        toolbox=toolbox,
        cxpb=0.5,          # crossover probability
        mutpb=0.2,         # mutation probability
        ngen=10,           # number of generations
        stats=stats,
        halloffame=hof,
        verbose=True
    )

    print("\nBest parameters found:", hof[0])
    return hof[0]

# Execute GA
best = run_deap()


gen	nevals	avg     	max     
0  	10    	0.782022	0.808989
1  	6     	0.791011	0.814607
2  	8     	0.796067	0.814607
3  	3     	0.802809	0.814607
4  	4     	0.81236 	0.814607
5  	4     	0.813483	0.820225
6  	10    	0.811236	0.820225
7  	9     	0.811236	0.820225
8  	6     	0.814607	0.820225
9  	7     	0.819663	0.820225
10 	8     	0.820225	0.820225

Best parameters found: [43, 4, 10]


In [22]:
# Final model using best hyperparameters
final_model = RandomForestClassifier(
    n_estimators=best[0],
    max_depth=best[1],
    min_samples_split=best[2],
    random_state=42
)

# Train the model
final_model.fit(X_train, y_train)

# Predict on test set
final_preds = final_model.predict(X_test)

# Evaluate accuracy
print("Final Test Accuracy:", accuracy_score(y_test, final_preds) * 100, "%")

Final Test Accuracy: 82.02247191011236 %
